In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
#from collections import OrderedDict
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#path="/home/anuja/Downloads/signature_dataset/signatures/"
#npzpath="/home/anuja/Downloads/signature_dataset/signatures/signature.npz"
img_height = 150
img_width = 220
featurewise_std_normalization = True
batch_size = 5    
epochs=20
seed=42
input_shape=(img_height, img_width,1)
lamb = 0.95
users=55
#checkpointer = ModelCheckpoint(filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5",verbose=1, save_best_only=False,save_weights_only=False,save_freq=1)
checkpointer = ModelCheckpoint(filepath="weights.hdf5",monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only=False)
#earlystoppetrain_file="/home/anuja/Downloads/signature_dataset/signatures/train_signature.npz"
#r = EarlyStopping(monitor='val_loss', min_delta=0,patience=4, verbose=1, mode='auto')
lr=0.001
lr_decay_multiplier=0.1
lr_decay_times=0.1
momentum=0.9
weight_decay=1e-4
epochs=20
seed=42
loss_type="L2"
input_size=(150,220)
#lamb in {0.95,0.99,0.999}
#checkpoint=#pth_file
lamb = 0.95
#checkpoint=#pth_file
users=55
#no_images=10
n_classes=55


In [1]:
import pandas as pd
data=pd.read_csv('signatures.csv')
#data=data.sample(frac=1).reset_index(drop=True)


In [ ]:
K.set_image_data_format('channels_last')
def train_model(input_shape):
    model=Sequential()
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_1', strides=(1,1), input_shape= input_shape, 
                    kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_2', strides=(1,1),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_3', strides=(1,1),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_4', strides=(2,2),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_5', strides=(2,2),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(ZeroPadding2D((2, 2),))
    model.add(Conv2D(256, (5, 5), activation='relu', name='conv2_1', strides=(1, 1),padding="same",kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(Dropout(0.3))# added extra
    #model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(384, (3, 3), activation='relu', name='conv3_1', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(384, (3, 3), activation='relu', name='conv3_2', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    #model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))   
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(Dropout(0.3))# added extra
    model.add(Flatten(name='flatten'))
    model.add(Dense(2048, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    #model.add(Dropout(0.5))
    model.add(Dense(2048, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')) # softmax changed to relu
    #last_features=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)
    return(model)

In [ ]:
input_image = Input(input_shape)
input_positve = Input(input_shape)
input_negative = Input(input_shape)
#X = Lambda(lambda  x: K.l2_normalize(x,axis=1))(X)

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):  
    anchor, positive, negative = y_pred[:,0,:], y_pred[:,1,:], y_pred[:,2,:]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive))
    neg_dist = tf.reduce_sum(tf.square(anchor - negative))
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.maximum(basic_loss, 0)
    return loss

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.patheffects as PathEffects

In [ ]:
x_train_flat = x_train.reshape(-1,1000)
tsne = TSNE()
train_tsne_embeds = tsne.fit_transform(x_train_flat)[:72]
scatter(train_tsne_embeds,data1["user"][:72], "Samples from Training Data")


In [ ]:
len(data1)
print(x_train.shape)

In [ ]:
x_train=[]
for i in range(72):
    img1=cv2.imread(data1.loc[i,"img"])
    img1 = cv2.resize(img1, (220,150))
    img1 = np.moveaxis(np.array([cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)]),0,-1)
    x_train.append(img1)
x_train=np.array(x_train)

In [ ]:
x_train.shape

In [ ]:
data1=data[data["yforg"]==0].reset_index()

In [ ]:
df = pd.DataFrame(columns=['Anchor', 'Positive', 'Negative'])
for digit in range(1):
    index=digit
    x=data[data["user"]==digit]
    x_true=x[x["yforg"]==0]
    x_forg=x[x["yforg"]==1]
    #print(list(permutations(x_true["img"],2)))
    #print(list(permutations(x_forg["img"],2)))
    permute=[]
    #output=list(permutations(x["img"],3))
    for i in range(len(x_true)):
        for j in range(i+1,len(x_true)):
            for k in range(len(x_forg)):
                df = df.append({'Anchor':x["img"][i+((index)*48)],'Positive':x["img"][j+((index)*48)],'Negative':x["img"][k+((index+1+index)*24)]},ignore_index=True)
    #print(permute,len(permute))
    #print(output[:10])
    #print(x_true,x_forg)

In [ ]:
df.iloc[169981]

In [ ]:
len(df)

In [ ]:
x["img"][k+(index*24)]

In [10]:
# import pandas
# df = pandas.DataFrame(data=permute)
df.to_csv("triplet_file_random_2.csv",index=False)


In [7]:
import random
df = pd.DataFrame(columns=['Anchor', 'Positive', 'Negative'])
for digit in range(55):
#     index=digit
    print(digit)
    random.seed(7)
    x=data[data["user"]==digit]
#     x_true=x[x["yforg"]==0]
#     x_forg=x[x["yforg"]==1]
    #print(list(permutations(x_true["img"],2)))
    #print(list(permutations(x_forg["img"],2)))
    permute=[]
    #output=list(permutations(x["img"],3))
    for i in range(600):
        ii=random.randrange(0,24)
        ji=random.randrange(0,24)
        ki=random.randrange(0,24)
        if ii!=ji:
            #print(ii,ji,ki)
            df = df.append({'Anchor':x["img"][ii+((digit)*48)],'Positive':x["img"][ji+((digit)*48)],'Negative':x["img"][ki+((digit+1+digit)*24)]},ignore_index=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [8]:
len(df)

31680

In [9]:
df

,Anchor,Positive,Negative
0,full_org/original_1_11.png,full_org/original_1_5.png,full_forg/forgeries_1_13.png
1,full_org/original_1_21.png,full_org/original_1_2.png,full_forg/forgeries_1_3.png
2,full_org/original_1_18.png,full_org/original_1_4.png,full_forg/forgeries_1_12.png
3,full_org/original_1_19.png,full_org/original_1_2.png,full_forg/forgeries_1_17.png
4,full_org/original_1_7.png,full_org/original_1_2.png,full_forg/forgeries_1_3.png
5,full_org/original_1_8.png,full_org/original_1_3.png,full_forg/forgeries_1_18.png
6,full_org/original_1_14.png,full_org/original_1_2.png,full_forg/forgeries_1_19.png
7,full_org/original_1_4.png,full_org/original_1_8.png,full_forg/forgeries_1_21.png
8,full_org/original_1_21.png,full_org/original_1_19.png,full_forg/forgeries_1_2.png
9,full_org/original_1_2.png,full_org/original_1_8.png,full_forg/forgeries_1_2.png
